<a href="https://www.kaggle.com/code/shoaibrkhan/data-expedition-movies?scriptVersionId=142798626" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import re

import warnings

warnings.filterwarnings("ignore")


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
movies = pd.read_csv("/kaggle/input/movies-dataset-for-feature-extracion-prediction/movies.csv")
df = pd.DataFrame(movies)
print(movies.shape)
movies.head()

In [ ]:
movies.info()

In [ ]:
movies.isnull().sum()

# Feature Extraction

In [ ]:
# As we can see the Gross column only have 460 non null values from 9539
df.drop('Gross', axis=1, inplace=True)

### Check For Duplicate Movies

In [ ]:
df['MOVIES'] = df['MOVIES'].str.strip()
duplicateData = df[df.duplicated(keep=False)]
duplicateData

In [ ]:
#we'll handle this dublicate movies later
duplicateData['MOVIES'].unique()

In [ ]:
# if the same movies (title) got diff ratings and votings then find the avg by them
agg_funcs = {
    'RATING': lambda x: x.fillna(0).astype(int).mean(),
    'VOTES': lambda x: x.str.replace(',', '').astype(float).fillna(0).astype(int).mean(),
}

for col in df.columns:
    if col not in agg_funcs and col != 'MOVIES':
        agg_funcs[col] = 'first'

df = df.groupby('MOVIES').agg(agg_funcs).reset_index()
df['VOTES'] = df['VOTES'].apply(lambda x: format(x, ','))

#also we are filling non null values of GENRE with 'Unknown'
df['GENRE'] = df['GENRE'].fillna('Unknown')
df

In [ ]:
# now we will do feature Extraction, clean the features in columns that include '\n'
df['GENRE'] = df['GENRE'].str.replace('\n', '')
df['ONE-LINE'] = df['ONE-LINE'].str.replace('\n', '')
df['STARS'] = df['STARS'].str.replace('\n', '')
df

In [ ]:
df['STARS']

In [ ]:
#as we can see our STARS column have both Director and Stars, now we'll split them in separate columns
def extract_names(row):
    directors = ', '.join(re.findall(r'Director[s]*:\s*([^|]+)', row))
    stars = ', '.join(re.findall(r'Stars*:\s*([^|]+)', row))
    return directors, stars

# Applying the function to create separate columns
df[['DIRECTOR', 'STARS']] = df['STARS'].apply(extract_names).apply(pd.Series)

# df['DIRECTOR'] = df['DIRECTOR'].str.strip()
# df['STARS'] = df['STARS'].str.strip()

df

In [ ]:
#Now we will be extracting text of STARS and Directors
df['DIRECTOR'] = df['DIRECTOR'].str.replace('Director:', '')
df['DIRECTOR'] = df['DIRECTOR'].str.replace('Directors:', '')
df['STARS'] = df['STARS'].str.replace('Stars:', '')

df

### Handling missing values In Years

#### We can add some other dataset related to this one so we can match them with the movies name and fill the missing years as we cannot remove them

In [ ]:
df[pd.isna(df['YEAR'])]

In [ ]:
imdb_movies = pd.read_csv("/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv")
imdb_df = pd.DataFrame(imdb_movies)
print(imdb_movies.shape)
imdb_movies.head()

In [ ]:
#the good thing is that, this data got 1000 movies as well and 1000 non null values in Released Year
#let's see how this dataset can help us
imdb_movies.info()

In [ ]:
#removing start and end spaces for movies in both dataset
df['MOVIES'] = df['MOVIES'].str.strip()
imdb_df['Series_Title'] = imdb_df['Series_Title'].str.strip()

In [ ]:
final_df = pd.merge(df, imdb_df[['Series_Title', 'Released_Year']], left_on='MOVIES', right_on='Series_Title', how='left')
final_df

In [ ]:
year_info = final_df[(final_df['Released_Year'].notna()) & (final_df['YEAR'].isna())]
year_info

In [ ]:
mask = final_df['Released_Year'].notna() & final_df['YEAR'].isna()
final_df.loc[mask, 'YEAR'] = final_df.loc[mask, 'Released_Year']

In [ ]:
final_df.isnull().sum()

In [ ]:
titles = pd.read_csv("/kaggle/input/netflix-tv-shows-and-movies/titles.csv")
titles_df = pd.DataFrame(titles)
print(titles.shape)
titles.head()

In [ ]:
titles_df.info()

In [ ]:
titles_df['title'] = titles_df['title'].str.strip()

In [ ]:
final_df = pd.merge(final_df, titles_df[['title', 'release_year']], left_on='MOVIES', right_on='title', how='left')
final_df

In [ ]:
year_info2 = final_df[(final_df['release_year'].notna()) & (final_df['YEAR'].isna())]
year_info2

In [ ]:
#removing the floats in release_year
final_df['release_year'] = final_df['release_year'].apply(lambda x: 'NaN' if pd.isna(x) else int(x))
# Convert 'release_year' to object type
final_df['release_year'] = final_df['release_year'].astype('object')

In [ ]:
final_df['release_year'] = final_df['release_year'].replace('NaN', pd.NA)

In [ ]:
final_df

In [ ]:
mask = final_df['release_year'].notna() & final_df['YEAR'].isna()
final_df.loc[mask, 'YEAR'] = final_df.loc[mask, 'release_year']

In [ ]:
final_df.isnull().sum()

In [ ]:
#dropping columns which we added
final_df = final_df.drop(columns=['Series_Title', 'Released_Year', 'title', 'release_year'])
final_df

#### We successfully replaced some of the years from our dataset which accurately matched by MOVIES
#### Now we also can use some fuzzy techniques as well here to match more items in from MOVIES but that would be more tricky

### Removing Duplicates

In [ ]:
#as we mentioned above, our movies data got so many duplicate movies, we will remove them
# Remove duplicates
# unique_df = final_df.drop_duplicates()
# unique_df

# a mask to identify rows where both 'YEAR' and 'Runtime' are NaN
mask_nan = final_df[['YEAR', 'RunTime']].isna().all(axis=1)

# a mask to keep rows that are not identified as NaN in both 'YEAR' and 'Runtime', 
# or if they are duplicate entries in the 'Movies' column (keeping the first occurrence)
mask_keep = ~mask_nan | final_df.duplicated(subset=['MOVIES'], keep='first')

# Apply the 'mask_keep' filter to the DataFrame
unique_df = final_df[mask_keep]

# Reset the index to ensure it reflects the updated DataFrame
unique_df = unique_df.reset_index(drop=True)
unique_df

In [ ]:
unique_df.isnull().sum()

In [ ]:
# checking duplicate again
duplicate_rows = unique_df[unique_df.duplicated(subset=['MOVIES'], keep=False)]
duplicate_rows

In [ ]:
#we will now simply drop_duplicate those entries
unique_df['MOVIES'] = unique_df['MOVIES'].str.strip()
unique_df = unique_df.drop_duplicates(subset=['MOVIES'], keep='first')
#checking again
duplicate_rows = unique_df[unique_df.duplicated(subset=['MOVIES'], keep=False)]
duplicate_rows

In [ ]:
movie_counts = unique_df['MOVIES'].value_counts()
movie_counts_df = movie_counts.reset_index()
movie_counts_df.columns = ['Movie', 'Count']
movie_counts_df.head(30)

In [ ]:
unique_df.isnull().sum()

In [ ]:
# we already cleaned duplicate movies on priority for NaN in YEAR and Runtime
# that helped us cleaned the YEAR data as well now we only got 13 null values, we'll try to
# fill this with fuzzy technique later
unique_df

# Exploratory Data Analysis

### Highest Rating

In [ ]:
top_10_movies = unique_df.sort_values(by='RATING', ascending=False).head(10)
# top_10_movies = unique_df.nlargest(10, 'RATING')
top_10_movies

### Most Votes on Movies

In [ ]:
new_df = unique_df.copy()
new_df['VOTES'] = new_df['VOTES'].str.replace(',', '').astype(float)
top_n_rows = new_df.nlargest(10, 'VOTES')
top_n_rows['VOTES'] = new_df['VOTES'].apply(lambda x: format(x, ','))
top_n_rows

## Finding Most Rated Movies By Year

In [ ]:
# movies data have NaN values on years so what we are doing now is to excluding data for that
# because we only got 13 null values so we can exclude that for now
unique_df.dropna(subset=['YEAR'], inplace=True)
unique_df

In [ ]:
#Removing the leading or trailing spaces from the genre 
unique_df['GENRE'] = unique_df['GENRE'].str.strip()
#separting Genre cell by commas
unique_genres = unique_df['GENRE'].str.split(', ').explode()
genre_counts = unique_genres.value_counts()
genre_counts

In [ ]:
unique_df.isnull().sum()

In [ ]:
#getting yearly highest rating movies
specific_year_data = unique_df[unique_df['YEAR'] == "(2020)"].sort_values(by='RATING', ascending=False).head(10)
specific_year_data

# Data Visualization

In [ ]:
sns.histplot(unique_df['RATING'])
plt.xlabel('RATING')
plt.title('Distribution of RATING')
plt.show()

In [ ]:
df_year = unique_df.copy()
df_year['RELEASE_YEAR_START'] = df_year['YEAR'].str.extract(r'(\d{4})')[0]
df_year['RELEASE_YEAR_END'] = df_year['YEAR'].str.extract(r'(\d{4})–(\d{4})')[1]

# Fill NaN values in RELEASE_YEAR_START and RELEASE_YEAR_END with '0' (for movies with single years)
df_year['RELEASE_YEAR_START'].fillna('0', inplace=True)
df_year['RELEASE_YEAR_END'].fillna('0', inplace=True)

df_year = df_year[(df_year['RELEASE_YEAR_START'].astype(int) >= 2000) & (df_year['RELEASE_YEAR_END'].astype(int) <= 2023)]

plt.figure(figsize=(10, 6))
sns.countplot(data=df_year, x='RELEASE_YEAR_START', palette='viridis')
plt.title('Number of Movies per Year', fontsize=16)
plt.xlabel('Release Year', fontsize=14)
plt.ylabel('Number of Movies', fontsize=14)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=top_10_movies, x='RATING', y='MOVIES', palette='viridis')
plt.title(f'Top Highest-Rated Movies', fontsize=16)
plt.xlabel('Rating', fontsize=14)
plt.ylabel('Movie Title', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
genre_counts.plot(kind='barh', color='skyblue')
plt.title('Genres in Movies', fontsize=16)
plt.xlabel('Count', fontsize=14)
plt.ylabel('Genre', fontsize=14)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
top_10_low_movies = unique_df.sort_values(by='RATING', ascending=False).tail(10)
plt.figure(figsize=(10, 6))
sns.barplot(data=top_10_low_movies, x='RATING', y='MOVIES', palette='viridis')
plt.title(f'Lowers-Rated Movies', fontsize=16)
plt.xlabel('Rating', fontsize=14)
plt.ylabel('Movie Title', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
numeric_df = df_year[['RATING', 'VOTES', 'RunTime']]
numeric_df['VOTES'] = numeric_df['VOTES'].str.replace(',', '').astype(float)
correlation_matrix = numeric_df.corr()

# Create a heatmap to visualize the correlation matrix
plt.figure(figsize=(8, 6))  # Set the figure size (optional)
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap', fontsize=16)  # Set the plot title (optional)
plt.show()